In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

In [2]:
# Primitive stochastic function 
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


In [6]:
# Define a simple stochastic function
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()
weather()

('sunny', 91.11004638671875)

In [8]:
# Turning PyTorch into Pyro 
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x.item())
# The crucial difference is that this sample is named. 
# Pyro’s backend uses these names to uniquely identify sample statements 
# and change their behavior at runtime depending 
# on how the enclosing stochastic function is being used. 

-0.08170700818300247


In [9]:
# Define a simple stochastic function in Pyro
def weather():
    cloudy = pyro.sample("second_sample",pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample("third_sample",pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()
weather()

('cloudy', 49.272254943847656)

In [11]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream.item()
ice_cream_sales()

tensor(73.4780)

In [15]:
# Because of Pyro's naming we now have recursion! 
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

1


In [16]:
# We are also free to define stochastic functions that accept as input or produce
# as output other stochastic functions:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(-0.8168)
